Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Category

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
response = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(response.content)

https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table 2')[0]

Loading in\data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 2']


In [4]:
observations = tab.excel_ref('B15').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)  


In [5]:
observations

{<L17 609.0>, <E19 '*'>, <L19 42.0>, <B20 88.4>, <M15 5969.0>, <D17 202.0>, <E16 '*'>, <B16 '*'>, <F19 '*'>, <B19 '*'>, <M21 3.1>, <I21 '*'>, <E20 81.3>, <M20 36.6>, <B15 95.0>, <K15 2036.0>, <G20 25.1>, <E15 713.0>, <F21 '*'>, <L20 44.9>, <G18 '*'>, <E21 '-'>, <M18 183.0>, <I20 30.6>, <J16 1719.0>, <D15 294.0>, <L21 13.1>, <E18 '-'>, <G21 '*'>, <L18 178.0>, <M17 2186.0>, <K16 1312.0>, <F20 31.0>, <H18 '*'>, <J18 5.0>, <E17 580.0>, <I15 5256.0>, <L15 1356.0>, <I17 1606.0>, <J17 853.0>, <L16 569.0>, <G15 1712.0>, <F15 2482.0>, <C15 324.0>, <H16 '*'>, <B17 84.0>, <J20 33.1>, <C20 90.7>, <C19 '*'>, <K18 0.0>, <H15 1062.0>, <K20 35.6>, <F18 '*'>, <B18 '-'>, <H20 38.3>, <I16 '*'>, <F16 '*'>, <J15 2577.0>, <B21 '-'>, <M16 3600.0>, <K17 724.0>, <G19 '*'>, <M19 60.3>, <H17 407.0>, <C16 '*'>, <K21 0.0>, <G16 '*'>, <H19 '*'>, <F17 769.0>, <I19 '*'>, <K19 64.4>, <H21 '*'>, <I18 '*'>, <J21 0.2>, <J19 66.7>, <G17 430.0>, <D16 '*'>, <C17 294.0>, <D19 '*'>, <D20 68.7>}

In [6]:
Service = tab.excel_ref('A15').expand(DOWN).is_not_blank()
Service

{<A21 'Prison (%)'>, <A17 'Non-statutory'>, <A20 'Non-statutory (%)'>, <A19 'Statutory (%)'>, <A15 'Total'>, <A16 'Statutory'>, <A18 'Prison'>}

In [7]:
Treatment = tab.excel_ref('B14').expand(RIGHT).is_not_blank()
Treatment

{<D14 'Drugs & Alcohol'>, <H14 'Drugs & Alcohol'>, <K14 'Drugs Only'>, <E14 'Total'>, <I14 'Total'>, <G14 'Drugs Only'>, <J14 'Alcohol Only'>, <F14 'Alcohol Only'>, <L14 'Drugs & Alcohol'>, <B14 'Alcohol Only'>, <C14 'Drugs Only'>}

In [8]:
age = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
age

{<M13 'Overall Total'>, <B13 'Under 18 '>, <F13 '18 and over'>, <J13 'Treatment Type'>}

In [9]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Category',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Period','2006-2016'),
            HDimConst('Sex','Persons')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Treatment Type,Category,Age,Measure Type,Unit,Period,Sex
0,95,NaN,Alcohol Only,Total,Under 18,Count,People,2006-2016,Persons
1,324,NaN,Drugs Only,Total,Under 18,Count,People,2006-2016,Persons
2,294,NaN,Drugs & Alcohol,Total,Under 18,Count,People,2006-2016,Persons
3,713,NaN,Total,Total,Under 18,Count,People,2006-2016,Persons
4,2482,NaN,Alcohol Only,Total,18 and over,Count,People,2006-2016,Persons
5,1712,NaN,Drugs Only,Total,18 and over,Count,People,2006-2016,Persons
6,1062,NaN,Drugs & Alcohol,Total,18 and over,Count,People,2006-2016,Persons
7,5256,NaN,Total,Total,18 and over,Count,People,2006-2016,Persons
8,2577,NaN,Alcohol Only,Total,Treatment Type,Count,People,2006-2016,Persons
9,2036,NaN,Drugs Only,Total,Treatment Type,Count,People,2006-2016,Persons


In [12]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [13]:
new_table.dtypes

Value             object
DATAMARKER        object
Treatment Type    object
Category          object
Age               object
Measure Type      object
Unit              object
Period            object
Sex               object
dtype: object

In [14]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Category,Age,Measure Type,Unit,Period,Sex
75,,*,Total,Prison (%),18 and over,Count,People,2006-2016,Persons
76,0.2,NaN,Alcohol Only,Prison (%),Treatment Type,Count,People,2006-2016,Persons
77,0,NaN,Drugs Only,Prison (%),Treatment Type,Count,People,2006-2016,Persons
78,13.1,NaN,Drugs & Alcohol,Prison (%),Treatment Type,Count,People,2006-2016,Persons
79,3.1,NaN,None,Prison (%),Overall Total,Count,People,2006-2016,Persons


In [15]:
new_table.count()

Value             80
DATAMARKER        28
Treatment Type    73
Category          80
Age               80
Measure Type      80
Unit              80
Period            80
Sex               80
dtype: int64

In [16]:
new_table = new_table[new_table['Value'] !=  0 ]

In [17]:
new_table = new_table[new_table['Value'] !=  '' ]

In [18]:
new_table.count()

Value             50
DATAMARKER         0
Treatment Type    43
Category          50
Age               50
Measure Type      50
Unit              50
Period            50
Sex               50
dtype: int64

In [19]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [20]:
new_table = new_table[['Period','Category','Age','Sex','Treatment Type','Measure Type','Value','Unit']]

In [21]:
new_table.head(5)

,Period,Category,Age,Sex,Treatment Type,Measure Type,Value,Unit
0,2006-2016,Total,Under 18,Persons,Alcohol Only,Count,95,People
1,2006-2016,Total,Under 18,Persons,Drugs Only,Count,324,People
2,2006-2016,Total,Under 18,Persons,Drugs & Alcohol,Count,294,People
3,2006-2016,Total,Under 18,Persons,Total,Count,713,People
4,2006-2016,Total,18 and over,Persons,Alcohol Only,Count,2482,People


In [22]:
# destinationFolder = Path('out')
# destinationFolder.mkdir(exist_ok=True, parents=True)

# new_table.to_csv(destinationFolder / ('tab2.2.csv'), index = False)